In [81]:
%load_ext autoreload

# trick for importing from different directory
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import statsmodels.api as sm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
import numpy as np
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff)

In [112]:
%autoreload 2
from wiki_forecasting.data_utils import prepare_data
X, y, sums, df = prepare_data()
X = X.loc[y<100]
sums = sums.loc[y<100]
y = y.loc[y<100]

In [113]:
X = X.drop(["date", "date_code"], axis=1).fillna(0)

In [114]:
import numpy as np
import pandas as pd
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.cov_struct import (Exchangeable,
    Independence,Autoregressive)
from statsmodels.genmod.families import Poisson


In [115]:
X["y"] = y

In [116]:
X.head()

,ewm,med1,med2,med3,median,page,y
Page,,,,,,,
2NE1_zh.wikipedia.org_all-access_spider,22.580592,14.0,14.0,19.0,15.0,0,13.0
2PM_zh.wikipedia.org_all-access_spider,34.413953,49.0,27.5,24.5,17.0,1,12.0
3C_zh.wikipedia.org_all-access_spider,5.250139,6.0,4.0,5.0,4.0,2,6.0
4minute_zh.wikipedia.org_all-access_spider,19.071762,13.0,16.0,14.5,12.0,3,38.0
52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,6.113785,7.0,5.5,4.0,4.0,4,13.0


In [120]:
ind = sm.cov_struct.Independence()
model = sm.GEE.from_formula("y ~ med1 + med2 + med3", "page",
                            data=X, family=sm.families.Poisson())
results = model.fit()
results.summary()

/usr/local/lib/python3.5/dist-packages/statsmodels/genmod/generalized_estimating_equations.py:1128: IterationLimitWarning: Iteration limit reached prior to convergence
  IterationLimitWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                               GEE Regression Results                              
===================================================================================
Dep. Variable:                           y   No. Observations:                48384
Model:                                 GEE   No. clusters:                      990
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                  50
Family:                            Poisson   Mean cluster size:                48.9
Dependence structure:         Independence   Num. iterations:                    60
Date:                     Thu, 31 Aug 2017   Scale:                           1.000
Covariance type:                    robust   Time:                         12:31:17
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -55.9055   6.07e+05   -9.2e-05      1.000   -1.19e+06    1.19e+06
med1           0.1265    162.560      0.001      0.999    -318.484     318.737
med2           0.1563   1575.984   9.91e-05      1.000   -3088.716    3089.029
med3           0.6232   9383.092   6.64e-05      1.000   -1.84e+04    1.84e+04
==============================================================================
Skew:                       -219.9568   Kurtosis:                   48379.0000
Centered skew:              -188.7544   Centered kurtosis:          40023.7636
==============================================================================
"""

In [121]:
print(results.predict())
print(X["median"].values)
print(smape(y, results.predict()))

[  3.82352073e-18   8.13901960e-14   4.73162499e-23 ...,   2.17663196e-21
   1.40863070e-19   2.89704823e-22]
[ 15.  17.   4. ...,   9.  14.   6.]
199.809496795


In [130]:
model = sm.GLM.from_formula("y ~ median + med1 + med2 + med3", X,
                           family=sm.families.Poisson())
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                48384
Model:                            GLM   Df Residuals:                    48379
Model Family:                 Poisson   Df Model:                            4
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:            -2.8214e+05
Date:                Thu, 31 Aug 2017   Deviance:                   3.5581e+05
Time:                        12:34:54   Pearson chi2:                 4.48e+05
No. Iterations:                     7                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.3422      0.002   1437.137      0.000       2.339       2.345
median         0.0041      0.000     39.580      0.000       0.004       0.004
med1           0.0031    6.5e-05     47.880      0.000       0.003       0.003
med2          -0.0033      0.000    -26.761      0.000      -0.004      -0.003
med3           0.0255      0.000    147.628      0.000       0.025       0.026
==============================================================================
"""

In [140]:
print(results.predict())
print(X["median"].values)
print(smape(y, results.predict()))

[ 17.94433942  22.21541917  12.08536876 ...,  13.67638081  15.57612378
  12.82031555]
[ 15.  17.   4. ...,   9.  14.   6.]
50.8094678606


In [125]:
model = sm.GLM.from_formula("y ~ median + med1 + med2 + med3", X,
                           family=sm.families.Gaussian())
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                48384
Model:                            GLM   Df Residuals:                    48379
Model Family:                Gaussian   Df Model:                            4
Link Function:               identity   Scale:                   139.089725397
Method:                          IRLS   Log-Likelihood:            -1.8804e+05
Date:                Thu, 31 Aug 2017   Deviance:                   6.7290e+06
Time:                        12:32:24   Pearson chi2:                 6.73e+06
No. Iterations:                     2                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.2329      0.091     46.710      0.000       4.055       4.411
median         0.3033      0.011     28.537      0.000       0.282       0.324
med1           0.1105      0.007     15.944      0.000       0.097       0.124
med2           0.1780      0.013     13.226      0.000       0.152       0.204
med3           0.3419      0.016     21.146      0.000       0.310       0.374
==============================================================================
"""

In [126]:
print(results.predict(), smape(y, results.predict()))

[ 19.31786512  28.07679388   8.53080756 ...,  12.7684853   17.24454753
  10.39824302] 46.8439557798
